<table style="border: none" align="left">
    <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/cars-4-you/master/static/images/logo.png" width="200" alt="Icon"></th>
       <th style="border: none"><font face="verdana" size="5" color="black"><b>Customer Satisfaction Prediction</b></th>
   </tr>
</table>

<img align=left src="https://github.com/pmservice/cars-4-you/raw/master/static/images/ai_function.png" alt="Icon" width="664">

Keras model and AI function to determine if comment is a complain.

Contents

- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Load and explore data](#load)
- [3. Create Keras model using TensorFlow backend](#model)
- [4. Store the model in the repository](#persistence)
- [5. Deploy the model](#deployment)
- [6. AI function](#ai_function)

<a id="setup"></a>
## 0. Setup

Install TensorFlow version 1.5 and newest version of watson-machine-learning-client.

In [6]:
!pip install --upgrade tensorflow==1.5

    100% |████████████████████████████████| 44.4MB 22kB/s  eta 0:00:01
Requirement not upgraded as not directly required: protobuf>=3.4.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
Requirement not upgraded as not directly required: numpy>=1.12.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
    100% |████████████████████████████████| 3.0MB 345kB/s eta 0:00:01
Requirement not upgraded as not directly required: six>=1.10.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
    100% |████████████████████████████████| 92kB 9.0MB/s eta 0:00:01
Requirement not upgraded as not directly required: wheel>=0.26 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from tensorflow==1.5)
Requirement not upgraded as not directly required: setuptools in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from protobuf>=3.4.0->tensorflow==1.5)
Requirement not upgraded as not direct

In [7]:
!rm -rf $PIP_BUILD/watson-machine-learning-client
!pip install --upgrade watson-machine-learning-client

    100% |████████████████████████████████| 942kB 1.1MB/s eta 0:00:01
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tqdm in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning

### WML Credentials

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**TIP:** Put your Watson Machine Learning credentials here.

In [4]:
# The code was removed by Watson Studio for sharing.

**TIP:** Put your Postgres credentials here.

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [7]:
client.version

'1.0.317'

### Clean up

In [8]:
for r in client.repository.get_model_details()['resources']:
    if r['entity']['name'] == 'CARS4U - Satisfaction Prediction Model':
        guid_to_delete = r['metadata']['guid']
        print('Deleting model: ' + str(guid_to_delete))
        client.repository.delete(guid_to_delete)

In [9]:
for r in client.repository.get_function_details()['resources']:
    if r['entity']['name'] == 'CARS4U - Satisfaction Prediction - AI Function':
        guid_to_delete = r['metadata']['guid']
        print('Deleting function: ' + str(guid_to_delete))
        client.repository.delete(guid_to_delete)

In [10]:
for r in client.deployments.get_details()['resources']:
    if r['entity']['name'] == 'CARS4U - Satisfaction Prediction - AI Function Deployment':
        guid_to_delete = r['metadata']['guid']
        print('Deleting function deployment: ' + str(guid_to_delete))
        client.deployments.delete(guid_to_delete)

<a id="introduction"></a>
## 1. Introduction

This notebook trains a **Keras** (TensorFlow) model to predict customer satisfaction based on provided feedback. Notebook also shows usage of **AI Function** for deep learning model data preprocessing required before model scoring.

<a id="load"></a>
## 2. Load and explore data

In this section the data is loaded as pandas dataframe.

In [11]:
# The code was removed by Watson Studio for sharing.

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,74,Male,M,1,26.26,Active,No,"no wait for pick up and drop off was great, he...",1,Product: Information,NA
1,83,Female,M,2,48.85,Inactive,Yes,I thought the representative handled the initi...,0,Product: Availability/Variety/Size,Free Upgrade
2,140,Female,S,0,36.92,Inactive,No,Everyone was very cooperative. The auto was r...,1,Product: Functioning,NA
3,191,Male,M,0,45.51,Inactive,Yes,what customer service? It was a nightmare,0,Service: Knowledge,Voucher
4,239,Male,M,1,46.00,Inactive,Yes,They did not have the auto I wanted. upgraded...,0,Product: Availability/Variety/Size,Free Upgrade


**Note:** 0 - not satisfied, 1 - satisfied

Extract needed columns and count number of records.

In [12]:
complain_data = data_df[['Customer_Service', 'Satisfaction']]

In [13]:
print(complain_data.count())

Customer_Service    482
Satisfaction        482
dtype: int64


<a id="model"></a>
## 3. Create Keras model using TensorFlow backend


In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import os
import numpy
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

### 3.1 Prepare data

In [15]:
max_fatures = 500

for idx,row in complain_data.iterrows():
    row[0] = row[0].replace('rt',' ')

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(complain_data['Customer_Service'].values)
X = tokenizer.texts_to_sequences(complain_data['Customer_Service'].values)

maxlen = 50

X = pad_sequences(X, maxlen=maxlen)
print(X.shape)

(482, 50)


Split into train and test datasets.

In [16]:
Y = complain_data['Satisfaction'].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(322, 50) (322,)
(160, 50) (160,)


### 3.2 Design and train model

Create the network definition based on Gated Recurrent Unit (Cho et al. 2014).

In [17]:
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(max_fatures, embedding_vector_length, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            16000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 72,405
Trainable params: 72,405
Non-trainable params: 0
_________________________________________________________________
None


Train the model.

In [18]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=64)

Train on 322 samples, validate on 160 samples
Epoch 1/20
322/322 [==============================] - 5s 14ms/step - loss: 0.6880 - acc: 0.5807 - val_loss: 0.6854 - val_acc: 0.5500
Epoch 2/20
322/322 [==============================] - 4s 12ms/step - loss: 0.6803 - acc: 0.5776 - val_loss: 0.6834 - val_acc: 0.5500
Epoch 3/20
322/322 [==============================] - 3s 10ms/step - loss: 0.6771 - acc: 0.5776 - val_loss: 0.6792 - val_acc: 0.5500
Epoch 4/20
322/322 [==============================] - 4s 11ms/step - loss: 0.6702 - acc: 0.5932 - val_loss: 0.6701 - val_acc: 0.5813
Epoch 5/20
322/322 [==============================] - 4s 12ms/step - loss: 0.6522 - acc: 0.6180 - val_loss: 0.6504 - val_acc: 0.6000
Epoch 6/20
322/322 [==============================] - 3s 11ms/step - loss: 0.6195 - acc: 0.7205 - val_loss: 0.6216 - val_acc: 0.6937
Epoch 7/20
322/322 [==============================] - 4s 11ms/step - loss: 0.5836 - acc: 0.8323 - val_loss: 0.5876 - val_acc: 0.7312
Epoch 8/20
322/322 [===

In [19]:
print("Best accuracy on test: %3.3f" % numpy.array(history.history['val_acc']).max())

Best accuracy on test: 0.919


**Note:** For purpose of this demo model tuning has been skipped.

Store and archive the model on notebook filesystem.

In [20]:
# evaluate the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Evaluation Accuracy: %.2f%%" % (scores[1]*100))

Evaluation Accuracy: 90.62%


In [21]:
print(scores[1])

0.90625


In [22]:
filename = 'complain_model.h5'
model.save(filename)

#compress keras model
tar_filename = filename + ".tgz"
cmdstring = "tar -zcvf " + tar_filename + " " + filename
os.system(cmdstring);

In [23]:
!ls -lat

total 1692
-rw-r----- 1 dsxuser dsxuser 818735 Aug 28 12:48 complain_model.h5.tgz
-rw-r----- 1 dsxuser dsxuser 903944 Aug 28 12:48 complain_model.h5
drwxr-x--- 2 dsxuser dsxuser   4096 Aug 28 06:56 .
drwx------ 1 dsxuser dsxuser   4096 Aug 28 06:55 ..


<a id="persistence"></a>
## 4. Store the model in the repository

In [24]:
score_accuracy = scores[1]
type(float(score_accuracy))

float

In [27]:
# model_props = {
#     client.repository.ModelMetaNames.NAME: "CARS4U - Action Recommendation Model",
#     client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
#     client.repository.ModelMetaNames.EVALUATION_METHOD: "multiclass",
#     client.repository.ModelMetaNames.EVALUATION_METRICS: [
#         {
#            "name": "accuracy",
#            "value": accuracy,
#            "threshold": 0.7
#         }
#     ]
# }


model_props = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Satisfaction Prediction Model tt",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5",
    client.repository.ModelMetaNames.RUNTIME_NAME: "python",
    client.repository.ModelMetaNames.RUNTIME_VERSION: "3.5",
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.1.3'}],
    client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    client.repository.ModelMetaNames.EVALUATION_METRICS: [{'name': 'accuracy', 'value': float(score_accuracy), 'threshold': 0.7 }],
    client.repository.ModelMetaNames.TRAINING_DEFINITION_URL: "https://us-south.ml.cloud.ibm.com/v3/ml_assets/training_definitions/ff24ebec-86af-4f2d-ab4c-2fc9e2a3efbc"
}

published_model_details = client.repository.store_model(model=tar_filename, meta_props=model_props)       

In [41]:
print(published_model_details)

{'metadata': {'created_at': '2018-08-28T12:51:12.290Z', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaed6937-c0e7-4307-8a17-361aca257c7e/published_models/2852162e-d279-4bd6-9f17-4d5fc7f2e90b', 'guid': '2852162e-d279-4bd6-9f17-4d5fc7f2e90b', 'modified_at': '2018-08-28T12:51:12.411Z'}, 'entity': {'model_type': 'tensorflow-1.5', 'name': 'CARS4U - Satisfaction Prediction Model tt', 'deployments': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaed6937-c0e7-4307-8a17-361aca257c7e/published_models/2852162e-d279-4bd6-9f17-4d5fc7f2e90b/deployments', 'count': 0}, 'evaluation_metrics_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaed6937-c0e7-4307-8a17-361aca257c7e/published_models/2852162e-d279-4bd6-9f17-4d5fc7f2e90b/evaluation_metrics', 'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/aaed6937-c0e7-4307-8a17-361aca257c7e/published_models/2852162e-d279-4bd6-9f17-4d5fc7f2e90b/learning_configuration', 'latest_version': {'cre

In [9]:
import requests

In [12]:
response = requests.get(url="https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/dad8cbb7-5563-428f-a905-9055145ce882", headers=client._get_headers())

In [13]:
response.text

'{"metadata":{"guid":"dad8cbb7-5563-428f-a905-9055145ce882","url":"https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/dad8cbb7-5563-428f-a905-9055145ce882","created_at":"2018-07-31T13:45:04.032Z","modified_at":"2018-08-06T13:18:12.992Z"},"entity":{"name":"CARS4U - Action Recommendation Model","model_url":"/v3/ml_assets/models/dad8cbb7-5563-428f-a905-9055145ce882","training_definition_url":"https://us-south.ml.cloud.ibm.com/v3/ml_assets/training_definitions/457fe9be-3ff5-4234-a12e-976f7304aaa4/versions/c8e9044b-0eb8-4948-b198-357d15436658","tags":[],"training_data_schema":{"type":"struct","fields":[{"name":"ID","type":"integer","nullable":true,"metadata":{"name":"ID","scale":0}},{"name":"Gender","type":"string","nullable":true,"metadata":{"name":"Gender","scale":0}},{"name":"Status","type":"string","nullable":true,"metadata":{"name":"Status","scale":0}},{"name":"Children","type":"integer","nullable":true,"metadata":{"name":"Children","scale":0}},{"name":"Age","type":"decimal(6,2)","n

In [8]:
client.repository.list_models()

------------------------------------  ------------------------------------------  ------------------------  --------------
GUID                                  NAME                                        CREATED                   FRAMEWORK
f1ce02df-317f-47a1-8c06-904e8435cbdd  Keras Model from Experiment                 2018-08-30T07:23:19.669Z  tensorflow-1.5
e5f8bc32-cb98-4ba3-a76e-2c715e4310db  testSaveModel                               2018-08-28T13:05:29.134Z  tensorflow-1.5
2852162e-d279-4bd6-9f17-4d5fc7f2e90b  CARS4U - Satisfaction Prediction Model tt   2018-08-28T12:51:12.290Z  tensorflow-1.5
eeb239c6-c503-4f02-bc9d-de2439e1ea51  Keras experiment model                      2018-08-28T11:53:38.538Z  tensorflow-1.5
d4be6ac1-eeb2-46ce-b510-eeef0c9dd2e6  ISSUE 5746 - Satisfaction Prediction Model  2018-08-27T10:15:25.650Z  tensorflow-1.5
dad8cbb7-5563-428f-a905-9055145ce882  CARS4U - Action Recommendation Model        2018-07-31T13:45:04.032Z  mllib-2.1
d928c3e2-8eb6-4d64-892a-e3

In [24]:
model_uid='f1ce02df-317f-47a1-8c06-904e8435cbdd'

In [25]:
spark_credentials = {
  "tenant_id": "s081-fcdcc2c8c4a157-70f20d2e11bc",
  "tenant_id_full": "9cb8e642-e850-49f8-9081-fcdcc2c8c4a1_5d5b82ce-01cb-4b9f-9c57-70f20d2e11bc",
  "cluster_master_url": "https://spark.bluemix.net",
  "tenant_secret": "7d6bb1ff-3965-4d41-8182-6156660e8194",
  "instance_id": "9cb8e642-e850-49f8-9081-fcdcc2c8c4a1",
  "plan": "ibm.SparkService.PayGoPersonal"
}
cos_credentials = {
  "apikey": "dCztud-nHdl11OxsiCMrqLmWKYLuhz4PUMs61KKX_iGZ",
  "cos_hmac_keys": {
    "access_key_id": "eec9bdf311bf4a7cb2cb168b5b5ea494",
    "secret_access_key": "e4b0d634c45aa29cf5adfaa37e6ad3cf7e15cdb6f7f51eaf"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/e0f7ec3ac1b24ec9ae771efd772538a2:4fd4d94f-61de-4046-91c1-0920eb5487d5::",
  "iam_apikey_name": "auto-generated-apikey-eec9bdf3-11bf-4a7c-b2cb-168b5b5ea494",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/e0f7ec3ac1b24ec9ae771efd772538a2::serviceid:ServiceId-fc100829-a31b-4157-a5f8-c857aefb2f53",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/e0f7ec3ac1b24ec9ae771efd772538a2:4fd4d94f-61de-4046-91c1-0920eb5487d5::"
}
feedback_data_reference = {
 "name": "Keras Model Experiment feedback",
 "connection": cos_credentials,
 "source": {
  "bucket": "keras-training-data-xy72"
 }
}

In [31]:
system_config = {
    client.learning_system.ConfigurationMetaNames.FEEDBACK_DATA_REFERENCE: feedback_data_reference,
    client.learning_system.ConfigurationMetaNames.MIN_FEEDBACK_DATA_SIZE: 10,
    client.learning_system.ConfigurationMetaNames.SPARK_REFERENCE: spark_credentials,
    client.learning_system.ConfigurationMetaNames.AUTO_RETRAIN: "always",
    client.learning_system.ConfigurationMetaNames.AUTO_REDEPLOY: "never"
}


In [32]:
learning_system_details = client.learning_system.setup(model_uid=model_uid, meta_props=system_config)

['AUTO_REDEPLOY',
 'AUTO_RETRAIN',
 'FEEDBACK_DATA_REFERENCE',
 'MIN_FEEDBACK_DATA_SIZE',
 'SPARK_REFERENCE']

In [33]:
print(learning_system_details)

{'feedback_data_reference': {'name': 'Keras Model Experiment feedback', 'source': {'bucket': 'keras-training-data-xy72'}, 'connection': {'iam_apikey_name': 'auto-generated-apikey-eec9bdf3-11bf-4a7c-b2cb-168b5b5ea494', 'iam_apikey_description': 'Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/e0f7ec3ac1b24ec9ae771efd772538a2:4fd4d94f-61de-4046-91c1-0920eb5487d5::', 'apikey': 'dCztud-nHdl11OxsiCMrqLmWKYLuhz4PUMs61KKX_iGZ', 'endpoints': 'https://cos-service.bluemix.net/endpoints', 'resource_instance_id': 'crn:v1:bluemix:public:cloud-object-storage:global:a/e0f7ec3ac1b24ec9ae771efd772538a2:4fd4d94f-61de-4046-91c1-0920eb5487d5::', 'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/e0f7ec3ac1b24ec9ae771efd772538a2::serviceid:ServiceId-fc100829-a31b-4157-a5f8-c857aefb2f53', 'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Manager', 'cos_hmac_keys': {'secret_access_key': 'e4b0d634c45aa29cf5adfaa37e6ad3cf7e

In [34]:
run_details = client.learning_system.run(model_uid, asynchronous=False)



#######################################################################

Synchronous run for uid: 'f8e56bbb-27be-4169-b2ac-863fcdd17f66' started

#######################################################################


RUNNING
ERROR

----------
Run failed
----------


Unexpected error during stage processing: execution is not specified neither in training definition nor in request


WMLClientError: Run failed. Errors: [{'message': 'Unexpected error during stage processing: execution is not specified neither in training definition nor in request', 'code': 'WMLDT0021E'}]

In [30]:
client.learning_system.list_runs(model_uid)

------------------------------------  ------------------------  -----
RUN GUID                              CREATED                   STATE
f259d106-7355-4a89-8772-4b6663c8dc08  2018-08-30T07:40:11.984Z  ERROR
------------------------------------  ------------------------  -----


In [25]:
client.repository.get_definition_details('ff24ebec-86af-4f2d-ab4c-2fc9e2a3efbc')

{'entity': {'author': {'name': 'WML WML-Beta'},
  'command': 'python keras_feedback.py',
  'framework': {'name': 'tensorflow', 'version': '1.5'},
  'name': 'Keras TD v4',
  'tags': [{'description': 'guid of associated DSX project',
    'value': 'dsx-project.cdaba989-5f37-4c01-a29a-e93b7038a62f'}],
  'training_definition_version': {'content_status': {'state': 'no_content'},
   'content_url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/training_definitions/ff24ebec-86af-4f2d-ab4c-2fc9e2a3efbc/versions/9fa00f2c-f7f2-4393-a846-7a8ab0d98b24/content',
   'guid': '9fa00f2c-f7f2-4393-a846-7a8ab0d98b24',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/training_definitions/ff24ebec-86af-4f2d-ab4c-2fc9e2a3efbc/versions/9fa00f2c-f7f2-4393-a846-7a8ab0d98b24'}},
 'metadata': {'created_at': '2018-08-28T11:46:16.961Z',
  'guid': 'ff24ebec-86af-4f2d-ab4c-2fc9e2a3efbc',
  'modified_at': '2018-08-28T11:46:17.030Z',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/training_definition

In [ ]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

<a id="deployment"></a>
## 5. Deploy the model

### 5.1 Create deployment

In [ ]:
deployment = client.deployments.create(model_uid, 'CARS4U - Satisfaction Prediction Model Deployment')

In [ ]:
client.deployments.list()

### 5.2 Score the model

Let's see if our deployment works.

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)

In [ ]:
print(scoring_endpoint)

In [ ]:
index = 5

scoring_data = X[index].tolist()
print(X_test[index])
print(Y_test[index])

In [ ]:
scoring_payload = {'values': [scoring_data]}
scores = client.deployments.score(scoring_endpoint, scoring_payload)

In [ ]:
print(scoring_payload)

In [ ]:
len(scoring_payload['values'][0])

In [ ]:
print({'values': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 266, 138, 139, 267, 207, 115, 12, 8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 2, 78, 5, 7, 13, 122, 3, 109, 51, 0, 0, 58, 15, 808, 31, 7, 23]]})

Let's print scoring results.

In [ ]:
print(str(scores))

<a id="function"></a>
## 6. AI function

Let's define AI function that does data preprocessing and model scoring for us. As noticed above model expects numerical input, so the text comment needs to be preprocessed.

### 6.1 Definition

Define some generic parameters our function will use to score the model.

#### Parameters

In [ ]:
ai_params = {
    'scoring_endpoint': scoring_endpoint,
    'wml_credentials': wml_credentials,
    'word_index': tokenizer.word_index,
    'nlu_url': 'https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze?version=2018-03-19',
    'nlu_username': '5c3be8c8-5932-4abe-bb39-7e7167602ccc',
    'nlu_password': 'cB4AYGkDB6rA'
}

In [ ]:
def score_generator(params=ai_params):

    def score_nlu(text):
        import requests

        payload = {"text": text, "features": {"sentiment": {}, "keywords": {} }}
        header = {"Content-Type":"application/json"}
        response_post = requests.post(params['nlu_url'], json=payload, headers=header, auth=(params['nlu_username'], params['nlu_password']))

        return(response_post.json()['sentiment']['document'])

    def score(payload):
        import re
        from watson_machine_learning_client import WatsonMachineLearningAPIClient
        client = WatsonMachineLearningAPIClient(params['wml_credentials'])
        
        max_fatures = 500
        maxlen = 50

        complain_data = payload['values']
        word_index = params['word_index']
        values = []
        
        for data in complain_data:
            comment = data[0]
            
            if len(comment) < 16:
                cleanString = re.sub(r"[!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~]", "", comment)
                splitted_comment = cleanString.split()[:maxlen]
                hashed_tokens = []

                for token in splitted_comment:
                    index = word_index.get(token, 0)
                    if index < 501 and index > 0:
                        hashed_tokens.append(index)

                hashed_tokens_size = len(hashed_tokens)
                padded_tokens = [0]*(maxlen-hashed_tokens_size) + hashed_tokens

                score_result = client.deployments.score(params['scoring_endpoint'], {'values':[padded_tokens]})
                values.append(score_result['values'][0])
            else:
                score_result = score_nlu(comment)
                score = score_result['score']
                predicted_class = score_result['label']
            
                if predicted_class == 'positive':
                    values.append([[score], [1], [score]])
                elif predicted_class == 'neutral':
                    values.append([[0.5 - score], [0], [0.5 - score]])
                else:
                    values.append([[score*-1], [0], [score*-1]])
        
        fields = ['prediction', 'prediction_classes', 'probability']

        return {'fields': fields, 'values': values}
        
        
    return score

#### Function

#### Test locally

In [ ]:
sample_data = {
    'fields': ['feedback'],
    'values': [
        ['delayed shuttle, almost missed flight, bad customer service'],
        ['The car was great and they were able to provide all features I wanted with limited time they had.'],
        ['The car was terrible but the service was good.'],
        ['I hate cars4you'],
        ['I love it.']
    ]
}

In [ ]:
score = score_generator()
score(sample_data)

**Note:** 0 - not satisfied. 1 - satisfied

### 6.2 AI function storing

In [ ]:
client.repository.FunctionMetaNames.show()

In [ ]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'CARS4U - Satisfaction Prediction - AI Function',
}

function_details = client.repository.store_function(meta_props=meta_data, function=score_generator)

In [ ]:
client.repository.list_functions()

### 6.3 AI function deployment

In [ ]:
function_uid = client.repository.get_function_uid(function_details)

function_deployment_details = client.deployments.create(artifact_uid=function_uid, name='CARS4U - Satisfaction Prediction - AI Function Deployment')

### Score AI function

In [ ]:
ai_function_scoring_endpoint = client.deployments.get_scoring_url(function_deployment_details)

print(ai_function_scoring_endpoint)

In [ ]:
response = client.deployments.score(ai_function_scoring_endpoint, sample_data)

In [ ]:
print(response)

---
